<a href="https://colab.research.google.com/github/deitar/JiraWiki2Docx/blob/main/JiraWiki2Docx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install xmltodict
# !pip install htmldocx
# !pip install python-docx

In [2]:
import os
import re
import html
import json
import xmltodict
import requests
from datetime import datetime
from requests.auth import HTTPBasicAuth
from docx import Document
from htmldocx import HtmlToDocx
from docx.shared import Inches
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.shared import Inches, RGBColor, Pt, Cm
from docx.enum.text import WD_LINE_SPACING
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from itertools import groupby
from datetime import datetime
import numpy as np
import pandas as pd
from enum import Enum

In [21]:
sample_jira_text = '''Carried out an {color:#ff5630}analysis to {color}{color:#fff0b3}identify{color}{color:#ff5630} customers who{color} carried out POS/WEB (card) transactions during the period of +^*-_cash scarcity but subsequently_-*^+ stopped transacting on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash scarcity.

*Definition of Terms*

* Cash scarcity period: November _*2022 to March 2023*_ (150 days)
** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)
* Period after cash scarcity: April 2023 to May 2023 (60 days)
**** Period after cash scarcity: April 2023 to May 2023 (60 days)

# test number 1
#* test number 11
#* test number 11
# test number 2
## test number 2

*Key Highlights:*

* Customers transactional behaviour before, during and after cash scarcity

||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarcity {color:#ff5630}AND {color}continued afterwards||
|Distinct Card Holders|10,674,893|{color:#ff5630}*310,454*{color}|1,002,235|

||* go
* be||
| |
| |


* *1.6 million (66%)* of the customers who completed card transactions during the cash scarcity period also carried out transactions prior to and after the cash scarcity period.
* *13% (1 million)* of the customers who carried out POS/WEB transactions during cash scarcity stopped card transactions at the end of the cash scarcity period
* Customers with POS/WEB transactions during cash scarcity but not after cash scarcity

||Period||Transacted prior to AND during cash scarcity BUT subsequently stopped||Transacted only during cash scarcity, NOT before or after cash scarcity||% of customers who transacted only during cash scarcity||
|Distinct Card Holders|{color:#36b37e}30,063 {color}|{color:#ff991f}20,391{color}|{color:#ff991f}41%{color}|

* *29% (3.03 of 11 million)* of these customers also carried out POS/WEB transactions in the 6 months prior to cash scarcity which means that they regularly transacted using their cards regardless of cash scarcity.-'''


In [22]:
class JiraWikiBlockTypes(Enum):
    LIST = 'list'
    HEADING = 'heading'
    TABLE = 'table'
    CODE = 'code'
    QUOTE = 'quote'
    PARAGRAPH = 'paragraph'

In [23]:
def detect_jira_block_type(jira_block):
    jira_block_trimmed = jira_block.strip()
    marker_heading = r'^h([1-6])\..*$'
    # marker_list = '^([\*|\-|\#]+)\s(.*)'
    marker_list = '^[*#-]+\s.*$'
    marker_table = '^[||]+.*$'
    if re.match(marker_heading, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.HEADING
    if re.match(marker_list, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.LIST
    if re.match(marker_table, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.TABLE
    return JiraWikiBlockTypes.PARAGRAPH

In [24]:
def get_jira_blocks(jira_text):
    return re.split('\n\n', jira_text)

In [25]:
def write_heading_jira_to_doc(text, doc_part_to_write_to):
    pattern_heading = r'^h([1-6])\.(.*)'
    heading_match = re.match(pattern_heading, text)
    heading_number, heading_text = heading_match.groups()
    para = doc_part_to_write_to.add_paragraph()
    para.style = f'Heading {heading_number}'
    # para.add_run(text=heading_text)
    apply_jira_text_effect_to_text_in_docx(heading_text, para)

In [26]:
def write_list_jira_to_doc(text, doc_part_to_write_to):
    pattern_list_item = r'^([\*|\-|\#]+)\s{1}(.*)'
    # split block into lines
    lines = re.split('\n', text)
    for line in lines:
        list_type, list_text = re.match(pattern_list_item, line).groups()
        # issue 1: couldn't resolve the round and square bullets differently
        # issue 2: find a way of decorating nested lists with different point styles
        # issue 3: sublevel list have unwanted top space to their parent list
        max_list_level = 3
        style_name = "List Number" if list_type[:max_list_level][-1] == '#' else 'List Bullet'
        style_name = f'{style_name} {str(min(len(list_type), max_list_level)) if len(list_type) > 1 else ""}'.strip()
        para = doc_part_to_write_to.add_paragraph()
        para.style = style_name
        # para.add_run(text = list_text)
        apply_jira_text_effect_to_text_in_docx(list_text, para)

In [27]:
def write_generic_jira_to_doc(text, doc_part_to_write_to):
    para = doc_part_to_write_to.add_paragraph()
    # para.add_run(text = text)
    apply_jira_text_effect_to_text_in_docx(text, para)

In [28]:
def write_jira_block_to_doc(jira_block):

    # handle leading new lines
    new_lines, actual_jira_block = re.match('(\n*)(.*)', jira_block, re.DOTALL).groups()
    num_new_lines = new_lines.count('\n')

    # add new lines by occurence
    for i in range(num_new_lines):
        para = doc.add_paragraph()
        para.add_run()

    # 1. heading
    if detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.HEADING:
        write_heading_jira_to_doc(actual_jira_block, doc)
    # 2. list
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.LIST:
        write_list_jira_to_doc(actual_jira_block, doc)
    # 3. table
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.TABLE:
        write_generic_jira_to_doc(actual_jira_block, doc)

    # any other
    else:
        write_generic_jira_to_doc(actual_jira_block, doc)

        # handle text effects



In [29]:
def detect_jira_text_effects(text, parent_key = None):
  text_effect_para_dict = {} # Dictionary to store detected text effects
  plain_text = text
  def detect_effects_recursive(text, parent_key=None):

    # Regular expressions for detecting various Jira text effects

    # Strong effect: Text enclosed in asterisks (*) with no space after the start or before the end.
    pattern_strong_effect = '((\*)(\S[^*]*\S)(\*))'

    # Italics effect: Text enclosed in underscores (_) with no space after the start or before the end.
    pattern_italics_effect = '((\_)(\S[^_]*\S)(\_))'

    # Deleted effect: Text enclosed in hyphens (-) with no space after the start or before the end.
    pattern_deleted_effect = '((\-)(\S[^-]*\S)(\-))'

    # Inserted effect: Text enclosed in plus signs (+) with no space after the start or before the end.
    pattern_inserted_effect = '((\+)(\S[^+]*\S)(\+))'

    # Superscript effect: Text enclosed in carets (^) with no space after the start or before the end.
    pattern_superscript_effect = '((\^)(\S[^\^]*\S)(\^))'

    # Subscript effect: Text enclosed in tildes (~) with no space after the start or before the end.
    pattern_subscript_effect = '((\~)(\S[^\~]*\S)(\~))'

    # Color effect: Text enclosed in color tags {color} with hexadecimal color code.
    pattern_color_effect = '((\{color:[#A-Za-z0-9]+\})(.*?)(\{color\}))'

    pattern_text_effect_list = [pattern_strong_effect,
                                pattern_italics_effect,
                                pattern_deleted_effect,
                                pattern_inserted_effect,
                                pattern_superscript_effect,
                                pattern_subscript_effect,
                                pattern_color_effect]
    pattern_text_effect = re.compile('|'.join(pattern_text_effect_list), re.DOTALL|re.IGNORECASE)
    for m in pattern_text_effect.finditer(text):
      try:
        full_match, text_effect_ope_tag, inner_text, text_effect_cls_tag = tuple(filter(lambda x: x, m.groups()))
        res = {}
        start_pos_outside_markup = m.start()
        end_pos_outside_markup = m.end() - 1 # decrement by 1 to match the end position in the string
        start_pos_inner_text = start_pos_outside_markup + len(text_effect_ope_tag)
        ope_tags = [text_effect_ope_tag]
        if parent_key:
          res = text_effect_para_dict.pop(parent_key)
          full_match = res['full_match']
          start_pos_outside_markup = res['start_pos_outside_markup']
          end_pos_outside_markup = res['end_pos_outside_markup']
          start_pos_inner_text = res['start_pos_inner_text'] + start_pos_inner_text
          ope_tags = res['ope_tags'] + ope_tags

        res['start_pos_outside_markup'] = start_pos_outside_markup
        res['end_pos_outside_markup'] = end_pos_outside_markup
        res['start_pos_inner_text'] = start_pos_inner_text
        res['end_pos_inner_text'] = res['start_pos_inner_text'] + len(inner_text) - 1
        res['inner_text'] = inner_text
        res['ope_tags'] = ope_tags
        res['full_match'] = full_match
        text_effect_para_dict[start_pos_outside_markup] = res
        detect_effects_recursive(inner_text, parent_key = start_pos_outside_markup)
      except Exception as e:
        print(e)
  detect_effects_recursive(text)
  return text_effect_para_dict


In [30]:
def apply_jira_text_effect_to_text_in_docx(text, docx_element):
  detected_text_effect_list = list(detect_jira_text_effects(text).values())
  prev_end_pos = -1
  for indx, item in enumerate(detected_text_effect_list):
    # apply
    inner_text = item['inner_text']
    start_pos_outside_markup = item['start_pos_outside_markup']
    end_pos_outside_markup = item['end_pos_outside_markup']
    start_pos_inner_text = item['start_pos_inner_text']
    end_pos_inner_text = item['end_pos_inner_text']
    ope_tags = item['ope_tags']

    leading_run_text = text[prev_end_pos + 1 : start_pos_outside_markup]
    current_run_text = text[start_pos_inner_text : end_pos_inner_text + 1]
    trailing_run_text = text[end_pos_outside_markup + 1:]

    leading_run = docx_element.add_run(leading_run_text)
    current_run = docx_element.add_run(current_run_text)

    # apply text effects in opening tags
    if '*' in ope_tags:
      current_run.bold = True
    if '_' in ope_tags:
      current_run.style = 'Emphasis'

    if indx == len(detected_text_effect_list) - 1:
      docx_element.add_run(trailing_run_text)

    prev_end_pos = end_pos_outside_markup


In [31]:
# Initialize a new Document
doc = Document()

In [32]:
for block in get_jira_blocks(sample_jira_text):
    print(repr(block))
    write_jira_block_to_doc(block)

'Carried out an {color:#ff5630}analysis to {color}{color:#fff0b3}identify{color}{color:#ff5630} customers who{color} carried out POS/WEB (card) transactions during the period of +^*-_cash scarcity but subsequently_-*^+ stopped transacting on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash scarcity.'
'*Definition of Terms*'
'* Cash scarcity period: November _*2022 to March 2023*_ (150 days)\n** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)\n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n**** Period after cash scarcity: April 2023 to May 2023 (60 days)'
'# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2'
'*Key Highlights:*'
'* Customers transactional behaviour before, during and after cash scarcity'
'||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped

In [33]:
doc.save('output.docx')

In [17]:
# doc = Document()
# para = doc.add_paragraph()
# test_block = '''Carried out an {color:#ff5630}analysis to {color}{color:#fff0b3}. identify{color}{color:#ff5630} customers who{color} carried out POS/WEB (card) transactions during the period of +^*-_cash scarcity but subsequently_-*^+ stopped transacting on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash scarcity.'''
# # detect_jira_text_effects(test_block)

In [18]:
# test_block = '''Carried out an {color:#ff5630}analysis to {color}{color:#fff0b3}. identify{color}{color:#ff5630} customers who{color} carried out POS/WEB (card) transactions during the period of +^*-_cash scarcity but subsequently_-*^+ stopped transacting on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash scarcity.'''
# detected_text_effect_list = list(detect_jira_text_effects(test_block).values())
# prev_end_pos = -1
# for indx, item in enumerate(detected_text_effect_list):
#   # apply
#   inner_text = item['inner_text']
#   start_pos_outside_markup = item['start_pos_outside_markup']
#   end_pos_outside_markup = item['end_pos_outside_markup']
#   start_pos_inner_text = item['start_pos_inner_text']
#   end_pos_inner_text = item['end_pos_inner_text']
#   ope_tags = item['ope_tags']

#   leading_run_text = test_block[prev_end_pos + 1 : start_pos_outside_markup]
#   current_run_text = test_block[start_pos_inner_text : end_pos_inner_text + 1]
#   trailing_run_text = test_block[end_pos_outside_markup + 1:]
#   leading_run = para.add_run(leading_run_text)
#   current_run = para.add_run(current_run_text)

#   # apply text effects in opening tags
#   if '*' in ope_tags:
#     current_run.bold = True
#   if '_' in ope_tags:
#     current_run.style = 'Emphasis'

#   if indx == len(detected_text_effect_list) - 1:
#     para.add_run(trailing_run_text)

#   prev_end_pos = end_pos_outside_markup


In [19]:
# doc.save('output.docx')

In [20]:
# ---- testing regex

test_block = '''C*ar*ried out an {color:#ff5630}analysis to {color}{color:#fff0b3}. identify{color}{color:#ff5630} customers who{color} carried out POS/WEB (card) transactions during the period of +^*-_cash scarcity but subsequently_-*^+ stopped transacting on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash scarcity.'''

pattern_strong_effect = '([*_-]+)(\S[^*_-]*\S)([*_-]+)'
# pattern_italics_effect = '(\_)(\S[^_]*\S)(\_)'
# pattern_color_effect = '(\{color:[A-Za-z]*\})(.?*)(\{color\})'
pattern_color_effect = '(\{color:[#A-Za-z0-9]+\})(.*?)(\{color\})'
pattern_text_effect = re.compile(f'{pattern_strong_effect}|{pattern_color_effect}', re.IGNORECASE | re.DOTALL)
# pattern_text_effect = re.compile(f'{pattern_color_effect}', re.IGNORECASE | re.DOTALL)
for m in pattern_text_effect.finditer(test_block):
    print(m.start(), '\t', m.groups(), '\t', m.end(), '\t') # test_block[m.start()])
    # print(list(filter(lambda x: x, m.groups())))

1 	 ('*', 'ar', '*', None, None, None) 	 5 	
17 	 (None, None, None, '{color:#ff5630}', 'analysis to ', '{color}') 	 51 	
51 	 (None, None, None, '{color:#fff0b3}', '. identify', '{color}') 	 83 	
83 	 (None, None, None, '{color:#ff5630}', ' customers who', '{color}') 	 119 	
183 	 ('*-_', 'cash scarcity but subsequently_', '-*', None, None, None) 	 219 	
250 	 ('*', 'channel and map out the', '*', None, None, None) 	 275 	
276 	 ('*_', 'transactional_', '*', None, None, None) 	 293 	
294 	 ('*', 'behaviour', '*', None, None, None) 	 305 	
309 	 ('_', 'these', '_', None, None, None) 	 316 	
317 	 ('*_', 'customers_', '*', None, None, None) 	 330 	
331 	 ('_', 'prior to and during', '_', None, None, None) 	 352 	
